In [2]:
%run read_file.ipynb

:: loading settings :: url = jar:file:/home/ec2-user/anaconda3/envs/python3/lib/python3.10/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/ec2-user/.ivy2/cache
The jars for the packages stored in: /home/ec2-user/.ivy2/jars
graphframes#graphframes added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-e68b2573-b5f6-4b7a-84f0-e3641f093703;1.0
	confs: [default]
	found graphframes#graphframes;0.8.2-spark3.2-s_2.12 in spark-packages
	found org.slf4j#slf4j-api;1.7.16 in central
downloading https://repos.spark-packages.org/graphframes/graphframes/0.8.2-spark3.2-s_2.12/graphframes-0.8.2-spark3.2-s_2.12.jar ...
	[SUCCESSFUL ] graphframes#graphframes;0.8.2-spark3.2-s_2.12!graphframes.jar (70ms)
downloading https://repo1.maven.org/maven2/org/slf4j/slf4j-api/1.7.16/slf4j-api-1.7.16.jar ...
	[SUCCESSFUL ] org.slf4j#slf4j-api;1.7.16!slf4j-api.jar (33ms)
:: resolution report :: resolve 698ms :: artifacts dl 111ms
	:: modules in use:
	graphframes#graphframes;0.8.2-spark3.2-s_2.12 from spark-packages in [default]
	org.slf4j#slf4j-api;1.7.16 from central in [default]
	-------

23/03/08 03:00:33 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


23/03/08 03:00:34 WARN SparkConf: Note that spark.local.dir will be overridden by the value set by the cluster manager (via SPARK_LOCAL_DIRS in mesos/standalone/kubernetes and LOCAL_DIRS in YARN).


In [7]:
%run build_adjacency.ipynb

 3:06AM UTC on Mar 08, 2023 --- read and cache time: 1.71661376953125e-05 seconds ---
root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [8]:
df_labels.printSchema()

root
 |-- hostname: string (nullable = true)
 |-- id: string (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- ts2: timestamp (nullable = true)



In [9]:
start_time = time.time()

test_dict = build_adjaceny(df_labels, 'labels')

print(time.strftime('%l:%M%p %Z on %b %d, %Y') + " --- " % (time.time() - start_time))

print(test_dict['d5846376-9596-4283-8505-8e6112abf1f2:e15537fe-fdb0-49d4-a1fd-de34e8215617'][0])

 3:08AM UTC on Mar 08, 2023 --- read and cache time: 33.56131649017334 seconds ---
000401b2-4413-4f26-9943-75877fd855df:3b7351a9-abe1-4e29-a7dc-18c0aafdb68c


In [10]:
df_eps = readCheckpoint()

 3:12AM UTC on Mar 08, 2023 --- read and cache time: 0.4459545612335205 seconds ---


In [12]:
df_eps.printSchema()

root
 |-- id: string (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- objectID: string (nullable = true)
 |-- actorID: string (nullable = true)
 |-- object: string (nullable = true)
 |-- action: string (nullable = true)
 |-- hostname: string (nullable = true)
 |-- user_name: string (nullable = true)
 |-- privileges: string (nullable = true)
 |-- image_path: string (nullable = true)
 |-- parent_image_path: string (nullable = true)
 |-- new_path: string (nullable = true)
 |-- file_path: string (nullable = true)
 |-- direction: string (nullable = true)
 |-- logon_id: string (nullable = true)
 |-- requesting_domain: string (nullable = true)
 |-- requesting_user: string (nullable = true)
 |-- event_minute: integer (nullable = true)
 |-- event_day: integer (nullable = true)
 |-- event_hour: integer (nullable = true)



In [16]:
print(df_eps.count())
df_eps.select("object").distinct().show()
df_eps.select("event_day").distinct().show()
df_eps.select("event_hour").distinct().show()

10000000
+-------+
| object|
+-------+
|  SHELL|
|   FLOW|
|   FILE|
|PROCESS|
+-------+

+---------+
|event_day|
+---------+
|       23|
+---------+

+----------+
|event_hour|
+----------+
|        16|
|        15|
|        17|
|        18|
|        13|
|        14|
|        19|
+----------+



In [18]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml import Pipeline

# Create a PySpark DataFrame
df = spark.createDataFrame([(0, "category1"), (1, "category2"), (2, "category3"), (3, "category1")], ["id", "category"])


In [19]:
df.toPandas()

,id,category
0,0,category1
1,1,category2
2,2,category3
3,3,category1


In [21]:
from pyspark.ml.feature import OneHotEncoder
from pyspark.ml.linalg import Vectors

# create sample data
data = [(0, "red"), (1, "green"), (2, "blue")]

# create dataframe
df = spark.createDataFrame(data, ["id", "color"])

# create OneHotEncoder object
encoder = OneHotEncoder(inputCols=["id"], outputCols=["id_vec"])

# fit and transform dataframe
df_encoded = encoder.fit(df).transform(df)

# show the encoded dataframe
df_encoded.show()

+---+-----+-------------+
| id|color|       id_vec|
+---+-----+-------------+
|  0|  red|(2,[0],[1.0])|
|  1|green|(2,[1],[1.0])|
|  2| blue|    (2,[],[])|
+---+-----+-------------+



In [ ]:
#create binary columns and then drop categorical column